In [ ]:
# Load required libraries for ArchR scATAC-seq analysis
library(ArchR)
library(parallel)
library(BSgenome.Mmusculus.UCSC.mm10)
library(ggpubr)
library(dplyr)
library(writexl)

# Set mouse mm10 genome for ArchR
addArchRGenome("mm10")

# Set random seed for reproducibility
set.seed(147)

Loading required package: ggplot2

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats

ERROR: Error in library(writexl): there is no package called ‘writexl’


# D30 Dataset

In [ ]:
# Load D30 ArchR project with single-cell peak set
proj <- loadArchRProject("/data/peer/sotougl/Fuchs/inflammatory_memory/scATAC/intermediate_outputs3/projects/ArchR_D30_min_NA_max_600_scpeakset")

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .____

In [ ]:
library(rtracklayer)
library(GenomicRanges)

# Import non-redundant peak set from bulk ATAC-seq analysis
bedData1 <- import.bed("/data/peer/sotougl/final_outs/peak_sets/all_nR_BWA_MACS3_replicable_D6D30M12_peaks.bed")

grangesData <- bedData1

# Filter for standard chromosomes only
grangesData <- grangesData[grep("^chr", seqnames(grangesData))]

# Exclude mitochondrial and Y chromosome peaks
grangesData <- grangesData[!(seqnames(grangesData) %in% c("chrM", "chrY"))]

# Add custom non-redundant peak set to ArchR project
proj = addPeakSet(ArchRProj = proj, peakSet = grangesData, force = TRUE)

# Disable HDF5 file locking to prevent multi-process conflicts
Sys.setenv(RHDF5_USE_FILE_LOCKING="FALSE")
Sys.setenv(HDF5_USE_FILE_LOCKING="FALSE")

# Generate peak matrix by counting fragments in each peak region
proj2 <- addPeakMatrix(proj)
peakm2 = getMatrixFromProject(proj2, useMatrix = 'PeakMatrix')

Warning message in min(abs(c(input%%1, input%%1 - 1)), na.rm = TRUE):
“no non-missing arguments to min; returning Inf”
ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-16242cda391b-Date-2025-01-05_Time-13-02-00.001141.log
If there is an issue, please report to github with logFile!

2025-01-05 13:02:00.456233 : Batch Execution w/ safelapply!, 0 mins elapsed.

.createArrowGroup : Arrow Group already exists! Dropping Group from ArrowFile! This will take ~10-30 seconds!

.dropGroupsFromArrow : Initializing Temp ArrowFile

.dropGroupsFromArrow : Adding Metadata to Temp ArrowFile

.dropGroupsFromArrow : Adding SubGroups to Temp ArrowFile

.dropGroupsFromArrow : Move Temp ArrowFile to ArrowFile

2025-01-05 13:02:21.78533 : Adding D30_PIMQ to PeakMatrix for Chr (1 of 20)!, 0.014 mins elapsed.

2025-01-05 13:02:28.126121 : Adding D30_PIMQ to PeakMatrix for Chr (2 of 20)!, 0.119 mins elapsed.

2025-01-05 13:02:34.814415 : Adding D30_PIMQ to PeakMatrix for Chr (3 of 20)!, 0.231 mins elapsed.

202

In [ ]:
# Save updated D30 project with non-redundant peak set
proj2 <- saveArchRProject(ArchRProj = proj2, outputDirectory = '/data/peer/sotougl/Fuchs/inflammatory_memory/scATAC/intermediate_outputs3/projects/ArchR_D30_min_NA_max_600_nonredundantpeakset')

Copying ArchRProject to new outputDirectory : /lila/data/peer/sotougl/Fuchs/inflammatory_memory/scATAC/intermediate_outputs3/projects/ArchR_D30_min_NA_max_600_nonredundantpeakset

Copying Arrow Files...

Copying Arrow Files (1 of 2)

Copying Arrow Files (2 of 2)

Getting ImputeWeights

No imputeWeights found, returning NULL

Copying Other Files...

Copying Other Files (1 of 9): Annotations

Copying Other Files (2 of 9): D30_Ctrl

Copying Other Files (3 of 9): D30_PIMQ

Copying Other Files (4 of 9): Embeddings

Copying Other Files (5 of 9): GroupCoverages

Copying Other Files (6 of 9): IterativeLSI

Copying Other Files (7 of 9): IterativeLSI2

Copying Other Files (8 of 9): PeakCalls

Copying Other Files (9 of 9): seacells

Saving ArchRProject...

Loading ArchRProject...

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\

# SEACell information

In [ ]:
# Set working directory to scripts folder
setwd('/data/peer/sotougl/Fuchs/inflammatory_memory/scATAC/Scripts')

In [ ]:
# Load D30 project with non-redundant peak set
proj <- loadArchRProject("/data/peer/sotougl/Fuchs/inflammatory_memory/scATAC/intermediate_outputs3/projects/ArchR_D30_min_NA_max_600_nonredundantpeakset")

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .____

In [ ]:
# Display cell counts per sample
data.frame(proj@cellColData) %>% group_by(Sample) %>% summarize(n = n())

Sample,n
<chr>,<int>
D30_Ctrl,5403
D30_PIMQ,5325


In [ ]:
# Export data for SEACells metacell construction in Python

# Create export directories
setwd(getOutputDirectory(proj))
dir.create('seacells')
dir.create("seacells/SEACells_export")
setwd("seacells/SEACells_export")

# Export LSI dimensionality reduction coordinates
write.csv(getReducedDims(proj, reducedDims = "IterativeLSI"), "svd.csv", quote = FALSE)

# Export cell metadata
write.csv(getCellColData(proj), "cell_metadata.csv", quote = FALSE)

# Export gene activity scores
gene.scores <- getMatrixFromProject(proj, useMatrix = "GeneScoreMatrix")
scores <- assays(gene.scores)[["GeneScoreMatrix"]]
scores <- as.matrix(scores)
rownames(scores) <- rowData(gene.scores)$name
write.csv(scores, "gene_scores.csv", quote = FALSE)

# Extract peak count matrix
peaks <- getPeakSet(proj)
peak.counts <- getMatrixFromProject(proj, "PeakMatrix")

# Reorder peaks by chromosome for consistent ordering
chr_order <- sort(seqlevels(peaks))
reordered_features <- list()
for(chr in chr_order){ reordered_features[[chr]] = peaks[seqnames(peaks) == chr] }
reordered_features <- Reduce("c", reordered_features)

# Export peak counts in sparse matrix format
wd <- getwd()
dir.create("peak_counts")
setwd("peak_counts")

counts <- assays(peak.counts)[["PeakMatrix"]]
writeMM(counts, "counts.mtx")
write.csv(colnames(peak.counts), "cells.csv", quote = FALSE)
names(reordered_features) <- sprintf("Peak%d", 1:length(reordered_features))
write.csv(as.data.frame(reordered_features), "peaks.csv", quote = FALSE)

Warning message in dir.create("seacells"):
“'seacells' already exists”
Warning message in dir.create("seacells/SEACells_export"):
“'seacells/SEACells_export' already exists”
ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-162422bfd1aa1-Date-2025-01-05_Time-13-08-28.888971.log
If there is an issue, please report to github with logFile!

2025-01-05 13:08:39.971349 : Organizing colData, 0.185 mins elapsed.

2025-01-05 13:08:40.000527 : Organizing rowData, 0.185 mins elapsed.

2025-01-05 13:08:40.003897 : Organizing rowRanges, 0.185 mins elapsed.

2025-01-05 13:08:40.008857 : Organizing Assays (1 of 1), 0.185 mins elapsed.

2025-01-05 13:08:40.113409 : Constructing SummarizedExperiment, 0.187 mins elapsed.

2025-01-05 13:08:40.795886 : Finished Matrix Creation, 0.198 mins elapsed.

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.9 GiB”
ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-162421ed89ccb-Date-2025-01-05_Time-13-11-59.486985

NULL

# Y1 Dataset

In [ ]:
# Load Year 1 ArchR project with single-cell peak set
proj <- loadArchRProject("/data/peer/sotougl/Fuchs/inflammatory_memory/scATAC/intermediate_outputs3/projects/ArchR_Y1_min_NA_max_600_scpeakset")

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .____

In [ ]:
library(rtracklayer)
library(GenomicRanges)

# Import non-redundant peak set from bulk ATAC-seq analysis
bedData1 <- import.bed("/data/peer/sotougl/final_outs/peak_sets/all_nR_BWA_MACS3_replicable_D6D30M12_peaks.bed")

grangesData <- bedData1

# Filter for standard chromosomes only
grangesData <- grangesData[grep("^chr", seqnames(grangesData))]

# Exclude mitochondrial and Y chromosome peaks
grangesData <- grangesData[!(seqnames(grangesData) %in% c("chrM", "chrY"))]

# Add custom non-redundant peak set to ArchR project
proj = addPeakSet(ArchRProj = proj, peakSet = grangesData, force = TRUE)

# Disable HDF5 file locking to prevent multi-process conflicts
Sys.setenv(RHDF5_USE_FILE_LOCKING="FALSE")
Sys.setenv(HDF5_USE_FILE_LOCKING="FALSE")

# Generate peak matrix by counting fragments in each peak region
proj2 <- addPeakMatrix(proj)
peakm2 = getMatrixFromProject(proj2, useMatrix = 'PeakMatrix')

Warning message in min(abs(c(input%%1, input%%1 - 1)), na.rm = TRUE):
“no non-missing arguments to min; returning Inf”
ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-162426961ebfe-Date-2025-01-05_Time-13-12-49.172279.log
If there is an issue, please report to github with logFile!

2025-01-05 13:12:49.29475 : Batch Execution w/ safelapply!, 0 mins elapsed.

.createArrowGroup : Arrow Group already exists! Dropping Group from ArrowFile! This will take ~10-30 seconds!

.dropGroupsFromArrow : Initializing Temp ArrowFile

.dropGroupsFromArrow : Adding Metadata to Temp ArrowFile

.dropGroupsFromArrow : Adding SubGroups to Temp ArrowFile

.dropGroupsFromArrow : Move Temp ArrowFile to ArrowFile

2025-01-05 13:13:06.334236 : Adding Y1_PIMQ to PeakMatrix for Chr (1 of 20)!, 0.013 mins elapsed.

2025-01-05 13:13:11.525617 : Adding Y1_PIMQ to PeakMatrix for Chr (2 of 20)!, 0.1 mins elapsed.

2025-01-05 13:13:17.284452 : Adding Y1_PIMQ to PeakMatrix for Chr (3 of 20)!, 0.196 mins elapsed.

2025-01

In [ ]:
# Save updated Year 1 project with non-redundant peak set
proj2 <- saveArchRProject(ArchRProj = proj2, outputDirectory = '/data/peer/sotougl/Fuchs/inflammatory_memory/scATAC/intermediate_outputs3/projects/ArchR_Y1_min_NA_max_600_nonredundantpeakset')

Copying ArchRProject to new outputDirectory : /lila/data/peer/sotougl/Fuchs/inflammatory_memory/scATAC/intermediate_outputs3/projects/ArchR_Y1_min_NA_max_600_nonredundantpeakset

Copying Arrow Files...

Copying Arrow Files (1 of 2)

Copying Arrow Files (2 of 2)

Getting ImputeWeights

No imputeWeights found, returning NULL

Copying Other Files...

Copying Other Files (1 of 9): Annotations

Copying Other Files (2 of 9): Embeddings

Copying Other Files (3 of 9): GroupCoverages

Copying Other Files (4 of 9): IterativeLSI

Copying Other Files (5 of 9): IterativeLSI2

Copying Other Files (6 of 9): PeakCalls

Copying Other Files (7 of 9): seacells

Copying Other Files (8 of 9): Y1_Ctrl

Copying Other Files (9 of 9): Y1_PIMQ

Saving ArchRProject...

Loading ArchRProject...

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\   

# SEACell Information

In [ ]:
# Set working directory to scripts folder
setwd('/data/peer/sotougl/Fuchs/inflammatory_memory/scATAC/Scripts')

In [ ]:
# Load Year 1 project with non-redundant peak set
proj <- loadArchRProject("/data/peer/sotougl/Fuchs/inflammatory_memory/scATAC/intermediate_outputs3/projects/ArchR_Y1_min_NA_max_600_nonredundantpeakset")

Successfully loaded ArchRProject!


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .____

In [ ]:
# Export data for SEACells metacell construction in Python

# Create export directories
setwd(getOutputDirectory(proj))
dir.create('seacells')
dir.create("seacells/SEACells_export")
setwd("seacells/SEACells_export")

# Export LSI dimensionality reduction coordinates
write.csv(getReducedDims(proj, reducedDims = "IterativeLSI"), "svd.csv", quote = FALSE)

# Export cell metadata
write.csv(getCellColData(proj), "cell_metadata.csv", quote = FALSE)

# Export gene activity scores
gene.scores <- getMatrixFromProject(proj, useMatrix = "GeneScoreMatrix")
scores <- assays(gene.scores)[["GeneScoreMatrix"]]
scores <- as.matrix(scores)
rownames(scores) <- rowData(gene.scores)$name
write.csv(scores, "gene_scores.csv", quote = FALSE)

# Extract peak count matrix
peaks <- getPeakSet(proj)
peak.counts <- getMatrixFromProject(proj, "PeakMatrix")

# Reorder peaks by chromosome for consistent ordering
chr_order <- sort(seqlevels(peaks))
reordered_features <- list()
for(chr in chr_order){ reordered_features[[chr]] = peaks[seqnames(peaks) == chr] }
reordered_features <- Reduce("c", reordered_features)

# Export peak counts in sparse matrix format
wd <- getwd()
dir.create("peak_counts")
setwd("peak_counts")

counts <- assays(peak.counts)[["PeakMatrix"]]
writeMM(counts, "counts.mtx")
write.csv(colnames(peak.counts), "cells.csv", quote = FALSE)
names(reordered_features) <- sprintf("Peak%d", 1:length(reordered_features))
write.csv(as.data.frame(reordered_features), "peaks.csv", quote = FALSE)

Warning message in dir.create("seacells"):
“'seacells' already exists”
Warning message in dir.create("seacells/SEACells_export"):
“'seacells/SEACells_export' already exists”
ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-162422b51e943-Date-2025-01-05_Time-13-17-57.959237.log
If there is an issue, please report to github with logFile!

2025-01-05 13:18:08.963493 : Organizing colData, 0.183 mins elapsed.

2025-01-05 13:18:08.992688 : Organizing rowData, 0.184 mins elapsed.

2025-01-05 13:18:08.996125 : Organizing rowRanges, 0.184 mins elapsed.

2025-01-05 13:18:09.001036 : Organizing Assays (1 of 1), 0.184 mins elapsed.

2025-01-05 13:18:09.130272 : Constructing SummarizedExperiment, 0.186 mins elapsed.

2025-01-05 13:18:09.873749 : Finished Matrix Creation, 0.199 mins elapsed.

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-1624265346d9c-Date-2025-01-05_Time-13-19-38.444416.log
If there is an issue, please report to github with logFile!

2025-01-05 13:20:01.395921 : Or

NULL

In [ ]:
# Display cell counts per sample
data.frame(proj@cellColData) %>% group_by(Sample) %>% summarize(n = n())

Sample,n
<chr>,<int>
Y1_Ctrl,2182
Y1_PIMQ,2294
